In [0]:
!java -version

openjdk version "1.8.0_275"
OpenJDK Runtime Environment (Zulu 8.50.0.51-CA-linux64) (build 1.8.0_275-b01)
OpenJDK 64-Bit Server VM (Zulu 8.50.0.51-CA-linux64) (build 25.275-b01, mixed mode)

In [0]:
sc.version

Out[2]: '3.1.0'

In [0]:
#!pip install mlflow

In [0]:
#import math
#import pandas as pd
#import matplotlib.pylab as plt
#%matplotlib inline

In [0]:
# carregar tabela de dados
df = table("cadastro_nomes_sexo")
#df = spark.sql("select * from cadastro_nomes_sexo")

In [0]:
df.printSchema()

root
-- PNOME: string (nullable = true)
-- SEXO: string (nullable = true)

In [0]:
df.show(10)

+---------+----+
 PNOME|SEXO|
+---------+----+
 MARIA| F|
 JOSE| M|
 ANA| F|
 JOAO| M|
 CARLOS| M|
 PAULO| M|
 ANTONIO| M|
 LUIZ| M|
FRANCISCO| M|
 MARCELO| M|
+---------+----+
only showing top 10 rows

In [0]:
rdd = df.rdd.filter(lambda x: x['SEXO'] in ['M', 'F'])
#rdd = df.rdd
#rdd = df.rdd.map(list)
#rdd = df.rdd.map(tuple)
rdd

Out[7]: PythonRDD[9] at RDD at PythonRDD.scala:58

In [0]:
type(rdd)

Out[8]: pyspark.rdd.PipelinedRDD

In [0]:
rdd.take(5)

Out[9]: [Row(PNOME='MARIA', SEXO='F'),
 Row(PNOME='JOSE', SEXO='M'),
 Row(PNOME='ANA', SEXO='F'),
 Row(PNOME='JOAO', SEXO='M'),
 Row(PNOME='CARLOS', SEXO='M')]

In [0]:
from pyspark.sql import Row

rdd2 = rdd.map(lambda x: Row(
  nome=x['PNOME'],
  sexo={'F': 0, 'M': 1, 'X': 9}[x['SEXO']]
))
rdd2

Out[10]: PythonRDD[11] at RDD at PythonRDD.scala:58

In [0]:
rdd2.take(5)

Out[11]: [Row(nome='MARIA', sexo=0),
 Row(nome='JOSE', sexo=1),
 Row(nome='ANA', sexo=0),
 Row(nome='JOAO', sexo=1),
 Row(nome='CARLOS', sexo=1)]

In [0]:
MAIOR_QTDE_LETRAS = 16 # fixado para o maior arquivo

def incluir_letras(row):
  row_dict = row.asDict()
  rev = row_dict['nome'][::-1]
  tam = len(rev)
  letras = [0] * MAIOR_QTDE_LETRAS
  
  for i in range(tam):
    letras[i] = ord(rev[i]) - 64
  row_dict['letras'] = letras
  
  new_row = Row(**row_dict)
  return new_row

In [0]:
rdd3 = rdd2.map(incluir_letras)
rdd3

Out[13]: PythonRDD[13] at RDD at PythonRDD.scala:58

In [0]:
rdd3.take(5)

Out[14]: [Row(nome='MARIA', sexo=0, letras=[1, 9, 18, 1, 13, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 Row(nome='JOSE', sexo=1, letras=[5, 19, 15, 10, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 Row(nome='ANA', sexo=0, letras=[1, 14, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 Row(nome='JOAO', sexo=1, letras=[15, 1, 15, 10, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 Row(nome='CARLOS', sexo=1, letras=[19, 15, 12, 18, 1, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])]

In [0]:
df3 = rdd3.toDF()
df3

Out[15]: DataFrame[nome: string, sexo: bigint, letras: array<bigint>]

In [0]:
df3.select("nome", "sexo").describe().show()

+-------+------+------------------+
summary| nome| sexo|
+-------+------+------------------+
 count| 41150| 41150|
 mean| NaN|0.4680194410692588|
 stddev| null|0.4989822586799242|
 min| AARAO| 0|
 max|ZYSMAN| 1|
+-------+------+------------------+

In [0]:
from pyspark.ml.linalg import Vectors

def converter_para_ponto_rotulado(row):
    obj = (row["nome"], row["sexo"], Vectors.dense(row["letras"]))
    return obj

In [0]:
rdd4 = rdd3.map(converter_para_ponto_rotulado)
rdd4

Out[18]: PythonRDD[26] at RDD at PythonRDD.scala:58

In [0]:
rdd4.take(5)

Out[19]: [('MARIA',
 0,
 DenseVector([1.0, 9.0, 18.0, 1.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])),
 ('JOSE',
 1,
 DenseVector([5.0, 19.0, 15.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])),
 ('ANA',
 0,
 DenseVector([1.0, 14.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])),
 ('JOAO',
 1,
 DenseVector([15.0, 1.0, 15.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])),
 ('CARLOS',
 1,
 DenseVector([19.0, 15.0, 12.0, 18.0, 1.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]))]

In [0]:
df4 = rdd4.toDF(["nome", "real", "features"])
df4

Out[20]: DataFrame[nome: string, real: bigint, features: vector]

In [0]:
df4.show(5, truncate=False)

+------+----+---------------------------------------------------------------------+
nome |real|features |
+------+----+---------------------------------------------------------------------+
MARIA |0 |[1.0,9.0,18.0,1.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0] |
JOSE |1 |[5.0,19.0,15.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0] |
ANA |0 |[1.0,14.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0] |
JOAO |1 |[15.0,1.0,15.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0] |
CARLOS|1 |[19.0,15.0,12.0,18.0,1.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]|
+------+----+---------------------------------------------------------------------+
only showing top 5 rows

In [0]:
from pyspark.ml.feature import PCA

pca = PCA(
  k = 10,
  inputCol = "features",
  outputCol = "pcaFeatures"
)

#pcaModel = pca.fit(df4)
#pcaResult = pcaModel.transform(df4).select("nome", "real", "pcaFeatures")
#pcaResult.show(5)

In [0]:
# usar redução de dimensionalidade PCA
#df5 = pcaResult.withColumnRenamed("pcaFeatures", "features")

# não usar PCA
df5 = df4

df5.show(5, truncate=False)

+------+----+---------------------------------------------------------------------+
nome |real|features |
+------+----+---------------------------------------------------------------------+
MARIA |0 |[1.0,9.0,18.0,1.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0] |
JOSE |1 |[5.0,19.0,15.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0] |
ANA |0 |[1.0,14.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0] |
JOAO |1 |[15.0,1.0,15.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0] |
CARLOS|1 |[19.0,15.0,12.0,18.0,1.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]|
+------+----+---------------------------------------------------------------------+
only showing top 5 rows

In [0]:
# dividir dados entre treino e teste
(dados_treino, dados_teste) = df5.randomSplit([0.7, 0.3], seed=42)

dados_treino.cache()
dados_teste.cache()

print("dados de treino:", dados_treino.count())
print("dados de teste: ", dados_teste.count())

dados de treino: 28895
dados de teste: 12255

In [0]:
# https://spark.apache.org/docs/latest/ml-statistics.html

'''from pyspark.ml.stat import Correlation

r1 = Correlation.corr(dados_treino, "features").head()
print("Pearson correlation matrix:\n" + str(r1[0]))

r2 = Correlation.corr(dados_treino, "features", "spearman").head()
print("Spearman correlation matrix:\n" + str(r2[0]))'''

Out[117]: 'from pyspark.ml.stat import Correlation\n\nr1 = Correlation.corr(dados_treino, "features").head()\nprint("Pearson correlation matrix:\n" + str(r1[0]))\n\nr2 = Correlation.corr(dados_treino, "features", "spearman").head()\nprint("Spearman correlation matrix:\n" + str(r2[0]))'

In [0]:
from pyspark.ml.stat import ChiSquareTest

r = ChiSquareTest.test(dados_treino, "features", "real").head()
print("pValues: " + str(r.pValues))
print("degreesOfFreedom: " + str(r.degreesOfFreedom))
print("statistics: " + str(r.statistics))

pValues: [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.637257105561844e-09,1.6843575534775113e-05,3.718938691055218e-05,0.03739793711488393,0.46830207592608697,0.32302180098266264,0.28777611678969917]
degreesOfFreedom: [20, 24, 26, 26, 26, 26, 26, 26, 26, 25, 25, 21, 16, 8, 2, 1]
statistics: [24932.73166384896,12661.439967337154,6324.827141049572,1186.5813205769834,713.8037043122233,733.1003573653618,314.1307721639996,222.472181892376,150.43242457758964,88.18921402675996,65.62013938285571,56.899226372573416,27.384930080638828,7.65080208702353,2.260070925469649,1.1299963530420603]

In [0]:
'''from pyspark.ml.stat import Summarizer

# create summarizer for multiple metrics "mean" and "count"
summarizer = Summarizer.metrics("mean", "count")

# compute statistics for multiple metrics with weight
dados_treino.select(summarizer.summary(dados_treino.features, dados_treino.real)).show(truncate=False)

# compute statistics for multiple metrics without weight
dados_treino.select(summarizer.summary(dados_treino.features)).show(truncate=False)

# compute statistics for single metric "mean" with weight
dados_treino.select(Summarizer.mean(dados_treino.features, dados_treino.real)).show(truncate=False)

# compute statistics for single metric "mean" without weight
dados_treino.select(Summarizer.mean(dados_treino.features)).show(truncate=False)'''

Out[116]: 'from pyspark.ml.stat import Summarizer\n\n# create summarizer for multiple metrics "mean" and "count"\nsummarizer = Summarizer.metrics("mean", "count")\n\n# compute statistics for multiple metrics with weight\ndados_treino.select(summarizer.summary(dados_treino.features, dados_treino.real)).show(truncate=False)\n\n# compute statistics for multiple metrics without weight\ndados_treino.select(summarizer.summary(dados_treino.features)).show(truncate=False)\n\n# compute statistics for single metric "mean" with weight\ndados_treino.select(Summarizer.mean(dados_treino.features, dados_treino.real)).show(truncate=False)\n\n# compute statistics for single metric "mean" without weight\ndados_treino.select(Summarizer.mean(dados_treino.features)).show(truncate=False)'

In [0]:
from datetime import datetime

models = {}

def evaluate_model(name, classifier, evaluator, train_data, test_data):

  start = datetime.now()
  model = classifier.fit(train_data)
  end = datetime.now()
  elapsed = int((end - start).total_seconds() * 1000)

  predictions = model.transform(test_data)
  score = evaluator.evaluate(predictions) * 100

  models[name] = (model, score, elapsed)
  print(model, '\nScore: %.2f [%5s ms]' % (score, elapsed))
  return model

In [0]:
# https://spark.apache.org/docs/latest/ml-tuning.html
import numpy as np
import re

from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

NUMBER_KFOLD_SPLITS = 5 # number of splits in cross-validation

# faz o ajuste fino do modelo, calculando os melhores hiperparâmetros
def fine_tune_model(estimator, params, evaluator, train_data, test_data):

  print('\nFine Tuning Model:')
  print(estimator) #, "\nparams:", params)
  
  crossval = CrossValidator(estimator=estimator,
                            estimatorParamMaps=params,
                            evaluator=evaluator,
                            numFolds=NUMBER_KFOLD_SPLITS,
                            seed=42,
                            parallelism=3)
  search = crossval.fit(train_data)

  predictions = search.transform(test_data)
  score = evaluator.evaluate(predictions) * 100

  hyper_list = []
  hyperparams = search.getEstimatorParamMaps()[np.argmax(search.avgMetrics)]
  for i in range(len(hyperparams.items())):
    hyper_name = re.search("name='(.+?)'", str([x for x in hyperparams.items()][i])).group(1)
    hyper_value = [x for x in hyperparams.items()][i][1]
    hyper_list.append({hyper_name: hyper_value})

  print('\nBest Score: %.2f' % score)
  print('Best Params:', hyper_list)
  return search

In [0]:
'''from pyspark.ml.evaluation import MulticlassClassificationEvaluator

avaliador = MulticlassClassificationEvaluator(
    predictionCol = "previsto",
    labelCol = "real",
    metricName = "accuracy"
)'''

Out[77]: 'from pyspark.ml.evaluation import MulticlassClassificationEvaluator\n\navaliador = MulticlassClassificationEvaluator(\n predictionCol = "previsto",\n labelCol = "real",\n metricName = "accuracy"\n)'

In [0]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

avaliador = BinaryClassificationEvaluator(
    rawPredictionCol = "previsto",
    labelCol = "real",
    metricName = "areaUnderROC"
)

In [0]:
# Logistic Regression
from pyspark.ml.classification import LogisticRegression

estimador = LogisticRegression(
  regParam=0.1,
  maxIter=10,
  labelCol="real",
  featuresCol="features",
  predictionCol="previsto"
)

params = ParamGridBuilder() \
    .addGrid(estimador.regParam, [1.0, 0.1, 0.01]) \
    .addGrid(estimador.maxIter, [10, 100, 1000]) \
    .build()
#fine_tune_model(estimador, params, avaliador, dados_treino, dados_teste)

evaluate_model('LR', estimador, avaliador, dados_treino, dados_teste)

LogisticRegressionModel: uid=LogisticRegression_13082fe0fd69, numClasses=2, numFeatures=16 
Score: 94.95 [ 1669 ms]
Out[81]: DataFrame[nome: string, real: bigint, features: vector, rawPrediction: vector, probability: vector, previsto: double]

In [0]:
# Decision Tree
from pyspark.ml.classification import DecisionTreeClassifier

estimador = DecisionTreeClassifier(
  maxDepth=9,
  labelCol="real",
  featuresCol="features",
  predictionCol="previsto"
)

params = ParamGridBuilder() \
    .addGrid(estimador.maxDepth, [2, 5, 7, 9, 11, 13]) \
    .build()
#fine_tune_model(estimador, params, avaliador, dados_treino, dados_teste)

evaluate_model('DT', estimador, avaliador, dados_treino, dados_teste)

DecisionTreeClassificationModel: uid=DecisionTreeClassifier_fcc58529b73c, depth=9, numNodes=199, numClasses=2, numFeatures=16 
Score: 99.62 [ 2393 ms]
Out[82]: DataFrame[nome: string, real: bigint, features: vector, rawPrediction: vector, probability: vector, previsto: double]

In [0]:
# Random Forest
from pyspark.ml.classification import RandomForestClassifier

estimador = RandomForestClassifier(
  maxDepth=19,
  labelCol="real",
  featuresCol="features",
  predictionCol="previsto"
)

params = ParamGridBuilder() \
    .addGrid(estimador.maxDepth, [9, 11, 13, 17, 19]) \
    .build()
#fine_tune_model(estimador, params, avaliador, dados_treino, dados_teste)

evaluate_model('RF', estimador, avaliador, dados_treino, dados_teste)

RandomForestClassificationModel: uid=RandomForestClassifier_3e168ff86bca, numTrees=20, numClasses=2, numFeatures=16 
Score: 99.19 [22776 ms]
Out[83]: DataFrame[nome: string, real: bigint, features: vector, rawPrediction: vector, probability: vector, previsto: double]

In [0]:
# Gradient-Boosted Trees (GBTs)
from pyspark.ml.classification import GBTClassifier

estimador = GBTClassifier(
  #maxDepth=10,
  labelCol="real",
  featuresCol="features",
  predictionCol="previsto"
)

params = ParamGridBuilder() \
    .addGrid(estimador.maxDepth, [5, 10]) \
    .build()
#fine_tune_model(estimador, params, avaliador, dados_treino, dados_teste)

# só classificação binária!
evaluate_model('GB', estimador, avaliador, dados_treino, dados_teste)

GBTClassificationModel: uid = GBTClassifier_b281d49decd1, numTrees=20, numClasses=2, numFeatures=16 
Score: 98.93 [18598 ms]
Out[103]: DataFrame[nome: string, real: bigint, features: vector, rawPrediction: vector, probability: vector, previsto: double]

In [0]:
# Multilayer Perceptron (MLP)
from pyspark.ml.classification import MultilayerPerceptronClassifier

estimador = MultilayerPerceptronClassifier(
  #maxIter=100,
  layers=[4, 5, 3],
  #blockSize=128,
  seed=42,
  labelCol="real",
  featuresCol="features",
  predictionCol="previsto"
)

params = ParamGridBuilder() \
    .addGrid(estimador.layers, [[4,5,3], [4,8,3], [4,5,6,3]]) \
    .build()
#fine_tune_model(estimador, params, avaliador, dados_treino, dados_teste)

# está dando erro...
#evaluate_model('MLP', estimador, avaliador, dados_treino, dados_teste)

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-3112302689744671> in <module> 
 18 
 19 # está dando erro... 
 ---> 20 evaluate_model ( 'MLP' , estimador , avaliador , dados_treino , dados_teste ) 

 <command-3112302689744656> in evaluate_model (name, classifier, evaluator, train_data, test_data) 
 11 
 12 predictions = model . transform ( test_data ) 
 ---> 13 score = evaluator . evaluate ( predictions ) * 100 
 14 
 15 models [ name ] = ( model , score , elapsed ) 

 /databricks/spark/python/pyspark/ml/evaluation.py in evaluate (self, dataset, params) 
 82 return self . copy ( params ) . _evaluate ( dataset ) 
 83 else : 
 ---> 84 return self . _evaluate ( dataset ) 
 85 else : 
 86 raise ValueError ( "Params must be a param map but got %s." % type ( params ) ) 

 /databricks/spark/python/pyspark/ml/evaluation.py in _evaluate (self, dataset) 
 118 """
 119 self . _transfer_params_to_java ( ) 
 --> 120 return self . _java_obj . evaluate ( dataset . _jdf ) 
 121 
 122 def isLargerBetter ( self ) : 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1302 
 1303 answer = self . gateway_client . send_command ( command ) 
 -> 1304 return_value = get_return_value(
 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 108 def deco ( * a , ** kw ) : 
 109 try : 
 --> 110 return f ( * a , ** kw ) 
 111 except py4j . protocol . Py4JJavaError as e : 
 112 converted = convert_exception ( e . java_exception ) 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 324 value = OUTPUT_CONVERTER [ type ] ( answer [ 2 : ] , gateway_client ) 
 325 if answer [ 1 ] == REFERENCE_TYPE : 
 --> 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 328 format(target_id, ".", name), value)

 Py4JJavaError : An error occurred while calling o98134.evaluate.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 46337.0 failed 1 times, most recent failure: Lost task 0.0 in stage 46337.0 (TID 44667) (ip-10-172-254-191.us-west-2.compute.internal executor driver): org.apache.spark.SparkException: Failed to execute user defined function(ProbabilisticClassificationModel$$Lambda$6257/1836602722: (struct<type:tinyint,size:int,indices:array<int>,values:array<double>>) => struct<type:tinyint,size:int,indices:array<int>,values:array<double>>)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anon$1.hasNext(WholeStageCodegenExec.scala:754)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:458)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:458)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:458)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:458)
	at org.apache.spark.util.collection.ExternalSorter.insertAll(ExternalSorter.scala:192)
	at org.apache.spark.shuffle.sort.SortShuffleWriter.write(SortShuffleWriter.scala:62)
	at org.apache.spark.shuffle.ShuffleWriteProcessor.write(ShuffleWriteProcessor.scala:59)
	at org.apache.spark.scheduler.ShuffleMapTask.runTask(ShuffleMapTask.scala:99)
	at org.apache.spark.scheduler.ShuffleMapTask.runTask(ShuffleMapTask.scala:52)
	at org.apache.spark.scheduler.Task.doRunTask(Task.scala:150)
	at org.apache.spark.scheduler.Task.run(Task.scala:119)
	at org.apache.spark.executor.Executor$TaskRunner.$anonfun$run$10(Executor.scala:732)
	at org.apache.spark.util.Utils$.tryWithSafeFinally(Utils.scala:1643)
	at org.apache.spark.executor.Executor$TaskRunner.run(Executor.scala:735)
	at java.util.concurrent.ThreadPoolExecuto

In [0]:
# Linear Support Vector Machine (SVM)
from pyspark.ml.classification import LinearSVC

estimador = LinearSVC(
  maxIter=100,
  regParam=0.1,
  labelCol="real",
  featuresCol="features",
  predictionCol="previsto"
)

params = ParamGridBuilder() \
    .addGrid(estimador.maxIter, [10, 50, 100, 500]) \
    .addGrid(estimador.regParam, [0.05, 0.1, 0.5]) \
    .build()
#fine_tune_model(estimador, params, avaliador, dados_treino, dados_teste)

# só classificação binária!
evaluate_model('LSVM', estimador, avaliador, dados_treino, dados_teste)

LinearSVCModel: uid=LinearSVC_6fc815743a5f, numClasses=2, numFeatures=16 
Score: 95.28 [ 9664 ms]
Out[84]: DataFrame[nome: string, real: bigint, features: vector, rawPrediction: vector, previsto: double]

In [0]:
# Naïve Bayes
from pyspark.ml.classification import NaiveBayes

estimador = NaiveBayes(
  smoothing=0.25,
  modelType="multinomial",
  labelCol="real",
  featuresCol="features",
  predictionCol="previsto"
)

params = ParamGridBuilder() \
    .addGrid(estimador.smoothing, [0.0, 0.25, 0.5, 0.75, 1.0]) \
    .addGrid(estimador.modelType, ["multinomial"]) \
    .build()
#fine_tune_model(estimador, params, avaliador, dados_treino, dados_teste)

evaluate_model('NB', estimador, avaliador, dados_treino, dados_teste)

NaiveBayesModel: uid=NaiveBayes_0a15e8b730ca, modelType=multinomial, numClasses=2, numFeatures=16 
Score: 91.98 [ 612 ms]
Out[85]: DataFrame[nome: string, real: bigint, features: vector, rawPrediction: vector, probability: vector, previsto: double]

In [0]:
# Factorization Machines (FM)
from pyspark.ml.classification import FMClassifier

estimador = FMClassifier(
  stepSize=0.001,
  labelCol="real",
  featuresCol="features",
  predictionCol="previsto"
)

params = ParamGridBuilder() \
    .addGrid(estimador.stepSize, [0.001, 0.01, 0.1, 1.0]) \
    .build()
#fine_tune_model(estimador, params, avaliador, dados_treino, dados_teste)

# só classificação binária!
evaluate_model('FM', estimador, avaliador, dados_treino, dados_teste)

FMClassificationModel: uid=FMClassifier_4bb846af56db, numClasses=2, numFeatures=16, factorSize=8, fitLinear=true, fitIntercept=true 
Score: 95.03 [33890 ms]
Out[86]: DataFrame[nome: string, real: bigint, features: vector, rawPrediction: vector, probability: vector, previsto: double]

In [0]:
#results = []
#for key, value in models.items():
#  tup = (key,) + value
#  results.append(tup)
#results

In [0]:
'''from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType
schema = StructType([ \
  StructField("Model", StringType(), True), \
  StructField("Estimator", StringType(), True), \
  StructField("Score", FloatType(), True), \
  StructField("Time (ms)", IntegerType(), True) \
])'''

Out[106]: 'from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType\nschema = StructType([ StructField("Model", StringType(), True), StructField("Estimator", StringType(), True), StructField("Score", FloatType(), True), StructField("Time (ms)", IntegerType(), True) ])'

In [0]:
'''results_df = spark.createDataFrame(data=results, schema=schema)
results_df.printSchema()
results_df.show(truncate=False)'''

Out[107]: 'results_df = spark.createDataFrame(data=results, schema=schema)\nresults_df.printSchema()\nresults_df.show(truncate=False)'

In [0]:
names = []
estimators = []
scores = []
times = []

for key, value in models.items():
  (model, score, elapsed) = value
  names.append(key)
  estimators.append(model)
  scores.append(score)
  times.append(elapsed)

In [0]:
import pandas as pd

results_df = pd.DataFrame({
  'Model': names,
  'Score': scores,
  'Time (ms)': times,
  'Estimator': estimators})

results_df.sort_values(by='Score', ascending=False)

Out[109]:

,Model,Score,Time (ms),Estimator
1,DT,99.615126,2393,DecisionTreeClassificationModel: uid=DecisionT...
2,RF,99.190351,22776,RandomForestClassificationModel: uid=RandomFor...
6,GB,98.934773,18598,GBTClassificationModel: uid = GBTClassifier_b2...
3,LSVM,95.275978,9664,"LinearSVCModel: uid=LinearSVC_6fc815743a5f, nu..."
5,FM,95.027094,33890,FMClassificationModel: uid=FMClassifier_4bb846...
0,LR,94.949332,1669,LogisticRegressionModel: uid=LogisticRegressio...
4,NB,91.981002,612,"NaiveBayesModel: uid=NaiveBayes_0a15e8b730ca, ..."


In [0]:
modelo = models['DT'][0]
modelo

Out[94]: DecisionTreeClassificationModel: uid=DecisionTreeClassifier_fcc58529b73c, depth=9, numNodes=199, numClasses=2, numFeatures=16

In [0]:
# gerar previsões e mostrar exemplos
previsoes = modelo.transform(dados_teste)
previsoes.select("nome", "real", "previsto").show(10)

+------------+----+--------+
 nome|real|previsto|
+------------+----+--------+
 AARONSON| 1| 1.0|
 ABADIO| 1| 1.0|
 ABAITE| 0| 0.0|
 ABDA| 0| 0.0|
 ABDALLAH| 0| 0.0|
 ABDEILDES| 1| 1.0|
ABDELMOUBINE| 0| 0.0|
 ABDIAS| 1| 1.0|
 ABDINARDO| 1| 1.0|
 ABDNAGOS| 1| 1.0|
+------------+----+--------+
only showing top 10 rows

In [0]:
display(previsoes.head(5))

nome,real,features,rawPrediction,probability,previsto
AARONSON,1,"Map(vectorType -> dense, length -> 16, values -> List(14.0, 15.0, 19.0, 14.0, 15.0, 18.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0))","Map(vectorType -> dense, length -> 2, values -> List(0.0, 2236.0))","Map(vectorType -> dense, length -> 2, values -> List(0.0, 1.0))",1.0
ABADIO,1,"Map(vectorType -> dense, length -> 16, values -> List(15.0, 9.0, 4.0, 1.0, 2.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0))","Map(vectorType -> dense, length -> 2, values -> List(1.0, 5894.0))","Map(vectorType -> dense, length -> 2, values -> List(1.6963528413910093E-4, 0.999830364715861))",1.0
ABAITE,0,"Map(vectorType -> dense, length -> 16, values -> List(5.0, 20.0, 9.0, 1.0, 2.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0))","Map(vectorType -> dense, length -> 2, values -> List(746.0, 8.0))","Map(vectorType -> dense, length -> 2, values -> List(0.9893899204244032, 0.010610079575596816))",0.0
ABDA,0,"Map(vectorType -> dense, length -> 16, values -> List(1.0, 4.0, 2.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0))","Map(vectorType -> dense, length -> 2, values -> List(7789.0, 0.0))","Map(vectorType -> dense, length -> 2, values -> List(1.0, 0.0))",0.0
ABDALLAH,0,"Map(vectorType -> dense, length -> 16, values -> List(8.0, 1.0, 12.0, 12.0, 1.0, 4.0, 2.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0))","Map(vectorType -> dense, length -> 2, values -> List(72.0, 0.0))","Map(vectorType -> dense, length -> 2, values -> List(1.0, 0.0))",0.0


In [0]:
# matriz de confusão
previsoes.groupBy("real", "previsto").count().show()

+----+--------+-----+
real|previsto|count|
+----+--------+-----+
 1| 0.0| 23|
 0| 1.0| 24|
 0| 0.0| 6538|
 1| 1.0| 5670|
+----+--------+-----+

In [0]:
previsoes.filter("real = 1").filter("real != previsto").select("nome", "real", "previsto").show(10)

+-------+----+--------+
 nome|real|previsto|
+-------+----+--------+
 ADONAI| 1| 0.0|
 ALIX| 1| 0.0|
 BRUCE| 1| 0.0|
 DESAIX| 1| 0.0|
 ELI| 1| 0.0|
 ELY| 1| 0.0|
 ESAULL| 1| 0.0|
 GEORGE| 1| 0.0|
 HERLY| 1| 0.0|
HERNANI| 1| 0.0|
+-------+----+--------+
only showing top 10 rows

In [0]:
previsoes.filter("real = 0").filter("real != previsto").select("nome", "real", "previsto").show(10)

+-------+----+--------+
 nome|real|previsto|
+-------+----+--------+
 ANNY| 0| 1.0|
 DORIS| 0| 1.0|
 ELENIR| 0| 1.0|
 GLADIS| 0| 1.0|
 GLEDSE| 0| 1.0|
 IASMIM| 0| 1.0|
 ISABEL| 0| 1.0|
 LADY| 0| 1.0|
 LEONOR| 0| 1.0|
LILLIAN| 0| 1.0|
+-------+----+--------+
only showing top 10 rows

In [0]:
# https://docs.databricks.com/applications/machine-learning/model-export/mleap-model-export.html#export-and-import-models-in-python
#!pip install mleap

In [0]:
%sh
rm -rf /tmp/mleap_python_model_export
mkdir /tmp/mleap_python_model_export

In [0]:
import mleap.pyspark
from mleap.pyspark.spark_support import SimpleSparkSerializer

modelo.serializeToBundle("jar:file:/tmp/mleap_python_model_export/72-modelo-genero-nome-pyspark-json.zip", previsoes)

--------------------------------------------------------------------------- 
 TypeError Traceback (most recent call last)
 <command-723065229944245> in <module> 
 2 from mleap . pyspark . spark_support import SimpleSparkSerializer
 3 
 ----> 4 modelo . serializeToBundle ( "jar:file:/tmp/mleap_python_model_export/72-modelo-genero-nome-pyspark-json.zip" , previsoes ) 

 /databricks/python/lib/python3.8/site-packages/mleap/pyspark/spark_support.py in serializeToBundle (self, path, dataset) 
 22 
 23 def serializeToBundle ( self , path , dataset = None ) : 
 ---> 24 serializer = SimpleSparkSerializer ( ) 
 25 serializer . serializeToBundle ( self , path , dataset = dataset ) 
 26 

 /databricks/python/lib/python3.8/site-packages/mleap/pyspark/spark_support.py in __init__ (self) 
 37 def __init__ ( self ) : 
 38 super ( SimpleSparkSerializer , self ) . __init__ ( ) 
 ---> 39 self . _java_obj = _jvm ( ) . ml . combust . mleap . spark . SimpleSparkSerializer ( ) 
 40 
 41 def serializeToBundle ( self , transformer , path , dataset ) : 

 TypeError : 'JavaPackage' object is not callable

In [0]:
dados = pd.DataFrame({'REAL': y, 'PREV': y_pred}, index=X.index)
for col in dados.columns:
    dados[col] = dados[col].map({0: 'X', 1: 'F', 2: 'M'})
dados.head()

In [0]:
from sklearn.metrics import accuracy_score

accuracy_score(y, y_pred)

In [0]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y, y_pred)

In [0]:
dados[dados['REAL'] != dados['PREV']].head(20)